#Problem 1(a).  Reading Amazon Reviews.

In this problem, we will analyze Amazon reviews to determine what characteristics make them most helpful.

Download the file of Amazon gourmet food reviews from the [Stanford Large Network Dataset Collection](https://snap.stanford.edu/data/web-FineFoods.html).   (Your computer may already have a utility installed that can unzip the archive as a text file; if not, [7-zip](http://www.7-zip.org/) is a useful utility for Windows. You can also use an online utility by doing a web search for: ``open .gz files online``.)

Create a pandas DataFrame object with the following entries for each review:

* Product ID
* Number of people who voted this review helpful
* Total number of people who rated this review
* Rating of the product
* Text of the review

For the second and third of these, the information will be given in the file as ```1/5```, which would correspond to 1 vote for helpful out of 5 people who rated the review.


In [1]:
import pandas as pd
import numpy as np

In [2]:
#Initially Loaded file to review data.
line_list=[]
#import file as list
with open("foods.txt") as f:
    line_num = 0
    for line in f:
        line=line.strip()
        line_list=f.read().splitlines()
print (len((line_list)))

line_list[0:20]




5116093


['review/userId: A3SGXH7AUHU8GW',
 'review/profileName: delmartian',
 'review/helpfulness: 1/1',
 'review/score: 5.0',
 'review/time: 1303862400',
 'review/summary: Good Quality Dog Food',
 'review/text: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.',
 '',
 'product/productId: B00813GRG4',
 'review/userId: A1D87F6ZCVE5NK',
 'review/profileName: dll pa',
 'review/helpfulness: 0/0',
 'review/score: 1.0',
 'review/time: 1346976000',
 'review/summary: Not as Advertised',
 'review/text: Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".',
 '',
 'product/productId: B000LQOCH0',
 'review/userId: ABXLMWJIXXAIN',
 'review/profileName: Natalia 

In [3]:
#Read in foods.txt file and strip out blank lines.  Did this in the datafile used to prevent having blanks in my dict below
rf = open("foods.txt")
wf = open("newfoods2.txt","w")
newlinecount=0
for line in rf:
    newline = line.strip('\r\n')
    if newline!= "":
        wf.write(newline)
        newlinecount=newlinecount+1
        wf.write('\n')  # remove to leave out line breaks
print (newlinecount)
rf.close()
wf.close()



4547639


In [4]:
#loaded lines into a dict.  Added line_number to key to create a unique key.  Did line splits utilizing the colon for the first  
#instance in the string to avoid issues with free text.
#Cleaned leading spaces using strip.  Left sample for review before applying to the entire file in the next frame.
#Dropped bad data rows (rows without prefix/data type identifiers).  Output the rows/data for review.
mydicta={}
with open("newfoods2.txt", 'r') as gourmet_file:
    line_num=0
    for line in gourmet_file:
        line = line.strip()
        line_num +=1
        if line_num <= newlinecount:
            if len(line.split(':', 1))!= 2:
                print (line, line_num)
            else:
                key=int(line_num)
                valuea, valueb = line.split(':', 1)
                valuea=valuea.strip()
                valueb=valueb.strip()
                mydicta[key] =valuea, valueb
        else:
            break

print(len(mydicta))

 
  



88 years old. ... 753580
...creative powers b... 1416685
School Princi... 1521590
School Princi... 2270671
I am a voracious reader/li... 2809464
School Princi... 3018833
...creative powers b... 4306898
4547632


#Problem 1(b).  Analyzing review text.

Add columns to your DataFrame for the length of a review, the number of exclamation points in a review, and the fraction of people who rated a review helpful. You should calculate the fraction who rated a review helpful using the two columns you made in 1a, and a ratio of 1 helpful rating out of 5 total ratings should be entered as 0.2, not the string ```1/5```. If no people voted on whether a problem was helpful, the corresponding entry in your percentage helpful column should be ```NaN```.

In [5]:
# create an empty data frame with the appropriate columns
column_names = ('Product', 'ReviewText', 'ReviewScore', 'HelpfulReviewCount', 'TotalReviewCount')
food_df = pd.DataFrame(columns=column_names)
    
#create separate lists of values and headers
myvaluesprod=[]
myvaluestext=[]
myvaluesscore=[]
myvalueshelp=[]
myvalueshelptotal=[]
for index, value in mydicta.items():
    if value[0] == ('product/productId'):
        myvaluesprod=myvaluesprod+[value[1]]
    if value[0] == 'review/text':
        myvaluestext=myvaluestext+[value[1]]
    if value[0] == 'review/score':
        myvaluesscore=myvaluesscore+[value[1]]
    if value[0] =='review/helpfulness':  
        myvalueshelp=myvalueshelp+[value[1].split("/")[0]]
        myvalueshelptotal=myvalueshelptotal+ [value[1].split("/")[1]]
d = {'Product' : myvaluesprod
    ,'ReviewText': myvaluestext
    ,'ReviewScore'  : (myvaluesscore)
    ,'HelpfulReviewCount' :  myvalueshelp
    ,'TotalReviewCount'    : myvalueshelptotal}

food_df = pd.DataFrame(d)
food_df.convert_objects(convert_numeric=True).dtypes



C:\Users\smc\Anaconda3\lib\site-packages\ipykernel\__main__.py:28: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


HelpfulReviewCount      int64
Product               float64
ReviewScore           float64
ReviewText             object
TotalReviewCount        int64
dtype: object

In [27]:
food_df = pd.DataFrame(d)

In [33]:
#Add columns to the dataframe.  Review Length, Exclamation Point Counts, Fraction of Reviewers who found the review helpful.  Convert objects to numeric

food_df=food_df.assign(ReviewLength=[len(value) for value in food_df['ReviewText']], ExclamCount=[value.count('!') for value in food_df['ReviewText']], 
FractionHelp=(food_df.HelpfulReviewCount.astype(int)/food_df.TotalReviewCount.astype(int).replace({ 0 : np.nan }))) 

food_df['ReviewScore']=pd.to_numeric(food_df['ReviewScore'])
food_df['TotalReviewCount']=pd.to_numeric(food_df['TotalReviewCount'])
food_df['HelpfulReviewCount']=pd.to_numeric(food_df['HelpfulReviewCount'])

#Problem 1(c).  Summary statistics.

How many reviews are in the data set?  What is the average length of a review (in characters)?  What is the average rating?  What is the greatest number of exclamation marks used in a single review?  Use the pandas package to answer these questions, then summarize your results in a markdown cell.

In [34]:
#Summary Statistics on all columns

food_df.describe(include='all')

,HelpfulReviewCount,Product,ReviewScore,ReviewText,TotalReviewCount,ExclamCount,FractionHelp,ReviewLength
count,568454.000000,568454,568454.000000,568454,568454.00000,568454.000000,298402.000000,568454.000000
unique,NaN,74258,NaN,393579,NaN,NaN,NaN,NaN
top,NaN,B007JFMH8M,NaN,"This review will make me sound really stupid, ...",NaN,NaN,NaN,NaN
freq,NaN,913,NaN,199,NaN,NaN,NaN,NaN
mean,1.743817,NaN,4.183199,NaN,2.22881,0.758047,0.776975,436.222083
std,7.636513,NaN,1.310436,NaN,8.28974,1.564842,0.346321,445.339741
min,0.000000,NaN,1.000000,NaN,0.00000,0.000000,0.000000,12.000000
25%,0.000000,NaN,4.000000,NaN,0.00000,0.000000,0.600000,179.000000
50%,0.000000,NaN,5.000000,NaN,1.00000,0.000000,1.000000,302.000000
75%,2.000000,NaN,5.000000,NaN,2.00000,1.000000,1.000000,527.000000


Number of Reviews in DataSet = 568454
Average Length of Review = 436.22
Average Rating = 4.18
Greatest Number of Exclamation Points Used in a Review = 84 

#Problem 1(d).  Export.

Save your DataFrame as a .csv file suitable for future analysis in R.  Your .csv file should not include the review text column, as the presence of commas and quotation marks will make reading the file difficult.  You should also convert entries from ```NaN``` to the empty string before saving.

In [8]:
# Save the data frame to a .csv file
# Specifying index=False prevents us from writing a column of row numbers

food_df.to_csv("food_df_clean.csv", columns=['Product', 'ReviewScore', 'HelpfulReviewCount', 'TotalReviewCount', 'ExclamCount', 'FractionHelp', 'ReviewLength'],
               na_rep="", index=False)